Em **NLP** a classificação é uma tarefa recorrente, como em:

- Identificar se uma avaliação nos comentários da sua loja é positiva ou negativa
- Identificar se um dado documento pertence a um determinado autor ou não
- Organização de documentos por tópicos/categorias

**Referências**

- [how make a table markdown](https://www.codecademy.com/resources/docs/markdown/tables)

- [class](https://www.youtube.com/watch?v=toUgBQv1BT8&list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU&index=4)

- [deberta](microsoft/deberta-v3-small)

- [model for finances analysis sentiment](https://huggingface.co/bardsai/finance-sentiment-pl-fast)

In [ ]:
%%capture
!pip install datasets
!pip install transformers[torch]

In [ ]:
# Mostrar arquivos
path = '/content'
!ls {path}

In [ ]:
# Import basics
import pandas as pd

trainPath = '/content/drive/MyDrive/Colab Notebooks/Dados/fastai/assets/train.csv'
train = pd.read_csv(trainPath)
train

Existem quadtro bibliotecas principais para "fazer" **Data Science** em Python.

|     |  |
| -------- | ------- |
| Numpy  | Numerical Operations    |
| Pandas      | DataFrame Manupulation     |
| Pytorch    | Neural Nets    |
| Matplotlib | Data Visualization


In [ ]:
# Descrição do dataset
train.describe(include='object')

In [ ]:
# Única coluna com todas as informações
train['input'] = 'TEXT1: ' + train.context + '; TEXT2: ' + train.target + '; ANC1: ' + train.anchor
train['input']

In [ ]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(train)
ds

**Neural Networks**
funcionam com números, elas não são capaz de entender letras, por esse motivo precisamos aplicar uma transformação chamada*Tokenização*.

In [ ]:
%%capture

## Chamar "tokenizador"
model_nm = 'microsoft/deberta-v3-small'

from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
# Demostração
tokz.tokenize("Meu nome é Carlos.")

In [ ]:
# Função para tokenizar um dataset
def tok_func(x): return tokz(x["input"])
tok_ds = ds.map(tok_func, batched=True)

In [ ]:
row = tok_ds[0]
row['input'], row['input_ids']

In [ ]:
# Transformers precisa que tenha uma coluna chamada "labels"
tok_ds = tok_ds.rename_columns({'score':'labels'})
tok_ds

Para avaliar o desempenho do nosso modelo vamos utilizar um conjunto de dados os quais o modelo não não no treino.

In [ ]:
evalPath = '/content/drive/MyDrive/Colab Notebooks/Dados/fastai/assets/test.csv'
eval_df = pd.read_csv(evalPath)
eval_df.describe()

In [ ]:
# Plotar função
def f(x): return -3*x**2 + 2*x + 20
import numpy as np, matplotlib.pyplot as plt

def plot_function(f, min=-2.1, max=2.1, color='r'):
    x = np.linspace(min,max, 100)[:,None]
    plt.plot(x, f(x), color)

plot_function(f)

In [ ]:
# Semente de reprodutibilidade
from numpy.random import normal,seed,uniform
np.random.seed(42)

In [ ]:
# Noise
def noise(x, scale): return normal(scale=scale, size=x.shape)
def add_noise(x, mult, add): return x * (1+noise(x,mult)) + noise(x,add)

In [ ]:
# Plot

x = np.linspace(-2, 2, num=20)[:,None]
y = add_noise(f(x), 0.2, 1.3)
plt.scatter(x,y);

In [ ]:
# Função para adequar aos dados
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

def plot_poly(degree):
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(x, y)
    plt.scatter(x,y)
    plot_function(model.predict)

In [ ]:
# Pouca precisão: Underfiting
plot_poly(1)

In [ ]:
# Aprendeu bem demais, overfiting

plot_poly(10)

In [ ]:
# Ideal
plot_poly(2)
plot_function(f, color='b')

In [ ]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

Como esperado precisamos de uma métrica para avaliar o desempenho do nosso modelos, vamos usar "r" ou **Pearson**

In [ ]:
# Housing Price
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing(as_frame=True)
housing = housing['data'].join(housing['target']).sample(1000, random_state=52)
housing.head()

In [ ]:
# Relação entre as vairáveis
np.set_printoptions(precision=2, suppress=True)
np.corrcoef(housing, rowvar=False)

In [ ]:
# Correlação entre duas colunas
def corr(x,y): return np.corrcoef(x,y)[0][1]
corr(housing.MedInc, housing.MedHouseVal)

In [ ]:
# Plotar correlação
def show_corr(df, a, b):
    x,y = df[a],df[b]
    plt.scatter(x,y, alpha=0.5, s=4)
    plt.title(f'{a} vs {b}; r: {corr(x, y):.2f}')

show_corr(housing, 'MedInc', 'MedHouseVal')

In [ ]:
show_corr(housing, 'MedInc', 'AveRooms')

In [ ]:
# Remover outliers
subset = housing[housing.AveRooms<15]
show_corr(subset, 'MedInc', 'AveRooms')

In [ ]:
# Calcular correlação em cada época
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [ ]:
from transformers import TrainingArguments,Trainer

bs = 128
epochs = 4

lr = 8e-5

args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

In [ ]:
# Treino
# trainer.train();

In [ ]:
# Previsão
# preds = trainer.predict(eval_ds).predictions.astype(float)
# preds